# Library Import

In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# Load Dataset

In [2]:
df = pd.read_csv("AirQuality_ModelReady.csv", index_col="timestamp", parse_dates=True)

# Preparation

In [3]:
pollutants = ["CO(GT)", "NMHC(GT)", "C6H6(GT)", "NOx(GT)", "NO2(GT)"]
horizons = [1, 6, 12, 24]
lags = [1, 2, 3, 24]

# CO Discretization

In [7]:
def is_CO_low(CO_val):
    if CO_val < 1.5:
        return 1

    return 0

def is_CO_mid(CO_val):
    if CO_val >= 1.5 and CO_val < 2.5:
        return 1

    return 0

def is_CO_high(CO_val):
    if CO_val >= 2.5:
        return 1

    return 0

def get_CO_class(CO_val):
    if is_CO_low(CO_val):
        return 0
    elif is_CO_mid(CO_val):
        return 1
    elif is_CO_high(CO_val):
        return 2

df_temp = df.copy()
df_temp["CO"] = df_temp["CO"].fillna(df_temp["CO"].mean())
df_temp["CO_low"] = df_temp["CO"].apply(is_CO_low)
df_temp["CO_mid"] = df_temp["CO"].apply(is_CO_mid)
df_temp["CO_high"] = df_temp["CO"].apply(is_CO_high)
df_temp["CO_class"] = df_temp["CO"].apply(get_CO_class)

for h in horizons:
    df_temp[f"CO_fut_{h}h"] = df_temp[f"CO_fut_{h}h"].fillna(df_temp[f"CO_fut_{h}h"].mean())
    df_temp[f"CO_fut_{h}h_low"] = df_temp[f"CO_fut_{h}h"].apply(is_CO_low)
    df_temp[f"CO_fut_{h}h_mid"] = df_temp[f"CO_fut_{h}h"].apply(is_CO_mid)
    df_temp[f"CO_fut_{h}h_high"] = df_temp[f"CO_fut_{h}h"].apply(is_CO_high)
    df_temp[f"CO_fut_{h}h_class"] = df_temp[f"CO_fut_{h}h"].apply(get_CO_class)

# Data Splitting

In [17]:
train = df_temp[df_temp.index.year == 2004]
test = df_temp[df_temp.index.year == 2005]

predictors = [
    "PT08.S1(CO)",
    "PT08.S2(NMHC)",
    "PT08.S3(NOx)",
    "PT08.S4(NO2)",
    "PT08.S5(O3)",
    "T",
    "RH",
    "AH",
    "hour",
    "weekday",
    "month",
    "CO_lag_1",
#    "CO_lag_3",
    "CO_lag_12",
    "CO_lag_24",
    "CO_roll_mean_3",
    "CO_roll_mean_6",
    "CO_roll_mean_12",
    "CO_roll_mean_24"
]

scaled_predictors = []

for predictor in predictors:
    scaled_predictors.append(predictor + "_scaled")

X_train = train[predictors]
X_test = test[predictors]

X_train_scaled = train[scaled_predictors]
X_test_scaled = test[scaled_predictors]

# Classification

## Decision Tree

In [33]:
dt_results = pd.DataFrame([], columns = ["horizon", "accuracy"])
for h in horizons:
    # Hyperparameters tuning
    y_train = train[f"CO_fut_{h}h_class"]

    tuning_train_ratio = 0.8
    tuning_train_test_split = int(len(X_train_scaled) * tuning_train_ratio)

    X_train_train = X_train.iloc[:tuning_train_test_split]
    y_train_train = y_train.iloc[:tuning_train_test_split]
    X_train_test = X_train.iloc[tuning_train_test_split:]
    y_train_test = y_train.iloc[tuning_train_test_split:]

    param_grid = {
        "max_depth": [3, 5, 7, 9, 11, 13, 15, None],
        "min_samples_leaf":[1, 5, 10, 15, 20, 25,],
        "max_features":    [None, "sqrt", "log2"],
    }
    
    best_params = None
    best_tuning_acc = -np.inf

    print(f"Tuning hyperparameters of decision tree for horizon {h}...")
    for max_depth in param_grid["max_depth"]:
        for min_leaf in param_grid["min_samples_leaf"]:
            for max_features in param_grid["max_features"]:
                dt_classifier = DecisionTreeClassifier(
                    max_depth = max_depth,
                    min_samples_leaf = min_leaf,
                    max_features = max_features,
                    random_state = 23
                )
                dt_classifier.fit(X_train_train, y_train_train)
                y_train_pred = dt_classifier.predict(X_train_test)
                tuning_acc = accuracy_score(y_train_test, y_train_pred)
    
                print(f"horizon={h}, depth={max_depth}, leaf={min_leaf}, max_features={max_features} => accuracy={tuning_acc:.4f}")
    
                if tuning_acc > best_tuning_acc:
                    best_tuning_acc = tuning_acc
                    best_params = {
                        "max_depth": max_depth,
                        "min_samples_leaf": min_leaf,
                        "max_features": max_features,
                    }
    
    print(f"For horizon: {h}, best params on validation:", best_params, "with accuracy:", best_tuning_acc)

    # Train and test the model with the tuned hyperparameters
    X_h_test_scaled = X_test_scaled.iloc[:(-1 * h)]
    y_h_test = test[f"CO_fut_{h}h_class"].iloc[:(-1 * h)]
    print(f"Decision Tree model training for horizon: {h}...")
    dt_classifier = DecisionTreeClassifier(
        max_depth = best_params["max_depth"],
        min_samples_leaf = best_params["min_samples_leaf"],
        max_features = best_params["max_features"],
        random_state = 23
    )
    start_training_time = time.perf_counter()
    dt_classifier.fit(X_train_scaled, y_train)
    end_training_time = time.perf_counter()
    training_time = end_training_time - start_training_time
    dt_pred = dt_classifier.predict(X_h_test_scaled)
    test_acc = accuracy_score(y_h_test, dt_pred)
    print(f"Finished using Decision Tree model for horizon: {h}! Training time: {training_time:.4f}s, Accuracy: {test_acc:.4f}\n\n")

    dt_results = pd.concat([dt_results, pd.DataFrame([{"horizon": h, "accuracy": test_acc}])], ignore_index=True)

print(dt_results)

Tuning hyperparameters of decision tree for horizon 1...
horizon=1, depth=3, leaf=1, max_features=None => accuracy=0.6547
horizon=1, depth=3, leaf=1, max_features=sqrt => accuracy=0.6828
horizon=1, depth=3, leaf=1, max_features=log2 => accuracy=0.6828
horizon=1, depth=3, leaf=5, max_features=None => accuracy=0.6547
horizon=1, depth=3, leaf=5, max_features=sqrt => accuracy=0.6828
horizon=1, depth=3, leaf=5, max_features=log2 => accuracy=0.6828
horizon=1, depth=3, leaf=10, max_features=None => accuracy=0.6547
horizon=1, depth=3, leaf=10, max_features=sqrt => accuracy=0.6828
horizon=1, depth=3, leaf=10, max_features=log2 => accuracy=0.6828
horizon=1, depth=3, leaf=15, max_features=None => accuracy=0.6547
horizon=1, depth=3, leaf=15, max_features=sqrt => accuracy=0.6828
horizon=1, depth=3, leaf=15, max_features=log2 => accuracy=0.6828
horizon=1, depth=3, leaf=20, max_features=None => accuracy=0.6547
horizon=1, depth=3, leaf=20, max_features=sqrt => accuracy=0.6828
horizon=1, depth=3, leaf=

/tmp/ipykernel_31287/1828838262.py:67: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dt_results = pd.concat([dt_results, pd.DataFrame([{"horizon": h, "accuracy": test_acc}])], ignore_index=True)


horizon=6, depth=3, leaf=15, max_features=None => accuracy=0.6041
horizon=6, depth=3, leaf=15, max_features=sqrt => accuracy=0.4423
horizon=6, depth=3, leaf=15, max_features=log2 => accuracy=0.4423
horizon=6, depth=3, leaf=20, max_features=None => accuracy=0.6041
horizon=6, depth=3, leaf=20, max_features=sqrt => accuracy=0.4423
horizon=6, depth=3, leaf=20, max_features=log2 => accuracy=0.4423
horizon=6, depth=3, leaf=25, max_features=None => accuracy=0.6041
horizon=6, depth=3, leaf=25, max_features=sqrt => accuracy=0.4423
horizon=6, depth=3, leaf=25, max_features=log2 => accuracy=0.4423
horizon=6, depth=5, leaf=1, max_features=None => accuracy=0.5781
horizon=6, depth=5, leaf=1, max_features=sqrt => accuracy=0.4437
horizon=6, depth=5, leaf=1, max_features=log2 => accuracy=0.4437
horizon=6, depth=5, leaf=5, max_features=None => accuracy=0.5802
horizon=6, depth=5, leaf=5, max_features=sqrt => accuracy=0.4437
horizon=6, depth=5, leaf=5, max_features=log2 => accuracy=0.4437
horizon=6, depth

## Support Vector Machine

In [36]:
svm_results = pd.DataFrame([], columns = ["horizon", "accuracy"])
for h in horizons:
    # Hyperparameters tuning
    y_train = train[f"CO_fut_{h}h_class"]

    tuning_train_ratio = 0.8
    tuning_train_test_split = int(len(X_train_scaled) * tuning_train_ratio)

    X_train_train = X_train.iloc[:tuning_train_test_split]
    y_train_train = y_train.iloc[:tuning_train_test_split]
    X_train_test = X_train.iloc[tuning_train_test_split:]
    y_train_test = y_train.iloc[tuning_train_test_split:]

    param_grid = {
        "C":     [0.1, 1, 10, 100, 1000],
        "gamma": ["scale", 0.1, 0.01, 0.001, 0.0001],
    }
    
    best_params = None
    best_tuning_acc = -np.inf

    print(f"Tuning hyperparameters of support vector machine for horizon {h}...")
    for C in param_grid["C"]:
        for gamma in param_grid["gamma"]:
            svm_classifier = SVC(
                kernel = "rbf",
                C = C,
                gamma = gamma,
                random_state = 23
            )
            svm_classifier.fit(X_train_train, y_train_train)
    
            y_train_pred = svm_classifier.predict(X_train_test)
            tuning_acc = accuracy_score(y_train_test, y_train_pred)
    
            print(f"horizon={h}, C={C}, gamma={gamma} => accuracy={tuning_acc:.4f}")
    
            if tuning_acc > best_tuning_acc:
                best_tuning_acc = tuning_acc
                best_params = {"C": C, "gamma": gamma}
    
    print(f"For horizon: {h}, best params on validation:", best_params, "with accuracy:", best_tuning_acc)

    # Train and test the model with the tuned hyperparameters
    X_h_test_scaled = X_test_scaled.iloc[:(-1 * h)]
    y_h_test = test[f"CO_fut_{h}h_class"].iloc[:(-1 * h)]

    print(f"Support Vector Machine model training for horizon: {h}...")
    svm_classifier = SVC(
        kernel = "rbf",
        C = best_params["C"],
        gamma = best_params["gamma"],
        random_state = 23
    )
    start_training_time = time.perf_counter()
    svm_classifier.fit(X_train_scaled, y_train)
    end_training_time = time.perf_counter()
    training_time = end_training_time - start_training_time
    svm_pred = dt_classifier.predict(X_h_test_scaled)
    test_acc = accuracy_score(y_h_test, svm_pred)
    print(f"Finished using Support Vector Machine model for horizon: {h}! Training time: {training_time:.4f}s, Accuracy: {test_acc:.4f}\n\n")

    svm_results = pd.concat([svm_results, pd.DataFrame([{"horizon": h, "accuracy": test_acc}])], ignore_index=True)

print(svm_results)

Tuning hyperparameters of support vector machine for horizon 1...
horizon=1, C=0.1, gamma=scale => accuracy=0.5640
horizon=1, C=0.1, gamma=0.1 => accuracy=0.3270
horizon=1, C=0.1, gamma=0.01 => accuracy=0.3270
horizon=1, C=0.1, gamma=0.001 => accuracy=0.3270
horizon=1, C=0.1, gamma=0.0001 => accuracy=0.3277
horizon=1, C=1, gamma=scale => accuracy=0.5837
horizon=1, C=1, gamma=0.1 => accuracy=0.3270
horizon=1, C=1, gamma=0.01 => accuracy=0.3270
horizon=1, C=1, gamma=0.001 => accuracy=0.3284
horizon=1, C=1, gamma=0.0001 => accuracy=0.3572
horizon=1, C=10, gamma=scale => accuracy=0.5893
horizon=1, C=10, gamma=0.1 => accuracy=0.3270
horizon=1, C=10, gamma=0.01 => accuracy=0.3340
horizon=1, C=10, gamma=0.001 => accuracy=0.3312
horizon=1, C=10, gamma=0.0001 => accuracy=0.3671
horizon=1, C=100, gamma=scale => accuracy=0.5985
horizon=1, C=100, gamma=0.1 => accuracy=0.3270
horizon=1, C=100, gamma=0.01 => accuracy=0.3439
horizon=1, C=100, gamma=0.001 => accuracy=0.3347
horizon=1, C=100, gamma=0.0

/tmp/ipykernel_31287/4139816952.py:63: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  svm_results = pd.concat([svm_results, pd.DataFrame([{"horizon": h, "accuracy": test_acc}])], ignore_index=True)


horizon=6, C=0.1, gamma=scale => accuracy=0.3284
horizon=6, C=0.1, gamma=0.1 => accuracy=0.3284
horizon=6, C=0.1, gamma=0.01 => accuracy=0.3284
horizon=6, C=0.1, gamma=0.001 => accuracy=0.3284
horizon=6, C=0.1, gamma=0.0001 => accuracy=0.3284
horizon=6, C=1, gamma=scale => accuracy=0.3284
horizon=6, C=1, gamma=0.1 => accuracy=0.3284
horizon=6, C=1, gamma=0.01 => accuracy=0.3284
horizon=6, C=1, gamma=0.001 => accuracy=0.3263
horizon=6, C=1, gamma=0.0001 => accuracy=0.3404
horizon=6, C=10, gamma=scale => accuracy=0.3284
horizon=6, C=10, gamma=0.1 => accuracy=0.3284
horizon=6, C=10, gamma=0.01 => accuracy=0.3298
horizon=6, C=10, gamma=0.001 => accuracy=0.3319
horizon=6, C=10, gamma=0.0001 => accuracy=0.3376
horizon=6, C=100, gamma=scale => accuracy=0.3354
horizon=6, C=100, gamma=0.1 => accuracy=0.3284
horizon=6, C=100, gamma=0.01 => accuracy=0.3404
horizon=6, C=100, gamma=0.001 => accuracy=0.3298
horizon=6, C=100, gamma=0.0001 => accuracy=0.3439
horizon=6, C=1000, gamma=scale => accuracy=

## Logistic Regression

In [39]:
lr_results = pd.DataFrame([], columns = ["horizon", "accuracy"])
for h in horizons:
    # Hyperparameters tuning
    y_train_low = train[f"CO_fut_{h}h_low"]
    y_train_mid = train[f"CO_fut_{h}h_mid"]
    y_train_high = train[f"CO_fut_{h}h_high"]
    y_train_class = train[f"CO_fut_{h}h_class"]

    tuning_train_ratio = 0.8
    tuning_train_test_split = int(len(X_train_scaled) * tuning_train_ratio)

    X_train_train = X_train_scaled.iloc[:tuning_train_test_split]
    y_train_train_low = y_train_low.iloc[:tuning_train_test_split]
    y_train_train_mid = y_train_mid.iloc[:tuning_train_test_split]
    y_train_train_high = y_train_high.iloc[:tuning_train_test_split]
    X_train_test = X_train_scaled.iloc[tuning_train_test_split:]
    y_train_test_low = y_train_low.iloc[tuning_train_test_split:]
    y_train_test_mid = y_train_mid.iloc[tuning_train_test_split:]
    y_train_test_high = y_train_high.iloc[tuning_train_test_split:]
    y_train_test_class = y_train_class.iloc[tuning_train_test_split:]

    param_grid = {
        "C":     [0.01, 0.1, 1, 10, 100, 1000]
    }
    
    best_params = None
    best_tuning_acc = -np.inf

    print(f"Tuning hyperparameters of logistic regression for horizon {h}...")
    for C in param_grid["C"]:
        lr_classifier_low = LogisticRegression(
            penalty = "l2",
            C = C,
            solver = "lbfgs",
            max_iter = 1000,
            random_state = 23
        )
        lr_classifier_mid = LogisticRegression(
            penalty = "l2",
            C = C,
            solver = "lbfgs",
            max_iter = 1000,
            random_state = 23
        )
        lr_classifier_high = LogisticRegression(
            penalty = "l2",
            C = C,
            solver = "lbfgs",
            max_iter = 1000,
            random_state = 23
        )

        lr_classifier_low.fit(X_train_train, y_train_train_low)
        lr_classifier_mid.fit(X_train_train, y_train_train_mid)
        lr_classifier_high.fit(X_train_train, y_train_train_high)

        train_proba_matrix = np.zeros((len(X_train_test), 3))
        train_proba_matrix[:, 0] = lr_classifier_low.predict_proba(X_train_test)[:, 1]
        train_proba_matrix[:, 1] = lr_classifier_mid.predict_proba(X_train_test)[:, 1]
        train_proba_matrix[:, 2] = lr_classifier_high.predict_proba(X_train_test)[:, 1]
    
        y_train_class_pred = np.argmax(train_proba_matrix, axis=1)
        tuning_acc = accuracy_score(y_train_test_class, y_train_class_pred)

        print(f"horizon={h}, C={C} => accuracy={tuning_acc:.4f}")

        if tuning_acc > best_tuning_acc:
            best_tuning_acc = tuning_acc
            best_params = {"C": C}
    
    print(f"For horizon: {h}, best params on validation:", best_params, "with accuracy:", best_tuning_acc)

    # Train and test the model with the tuned hyperparameters
    X_h_test_scaled = X_test_scaled.iloc[:(-1 * h)]
    y_h_class_test = test[f"CO_fut_{h}h_class"].iloc[:(-1 * h)]

    print(f"Logistic Regression model training for horizon: {h}...")
    lr_classifier_low = LogisticRegression(
        penalty = "l2",
        C = best_params["C"],
        solver = "lbfgs",
        max_iter = 1000,
        random_state = 23
    )
    lr_classifier_mid = LogisticRegression(
        penalty = "l2",
        C = best_params["C"],
        solver = "lbfgs",
        max_iter = 1000,
        random_state = 23
    )
    lr_classifier_high = LogisticRegression(
        penalty = "l2",
        C = best_params["C"],
        solver = "lbfgs",
        max_iter = 1000,
        random_state = 23
    )

    start_training_time = time.perf_counter()
    lr_classifier_low.fit(X_train_scaled, y_train_low)
    lr_classifier_mid.fit(X_train_scaled, y_train_mid)
    lr_classifier_high.fit(X_train_scaled, y_train_high)
    end_training_time = time.perf_counter()
    training_time = end_training_time - start_training_time

    test_proba_matrix = np.zeros((len(X_h_test_scaled), 3))
    test_proba_matrix[:, 0] = lr_classifier_low.predict_proba(X_h_test_scaled)[:, 1]
    test_proba_matrix[:, 1] = lr_classifier_mid.predict_proba(X_h_test_scaled)[:, 1]
    test_proba_matrix[:, 2] = lr_classifier_high.predict_proba(X_h_test_scaled)[:, 1]

    y_train_class_pred = np.argmax(test_proba_matrix, axis=1)
    test_acc = accuracy_score(y_h_class_test, y_train_class_pred)
    print(f"Finished using Logistic Regression model for horizon: {h}! Training time: {training_time:.4f}s, Accuracy: {test_acc:.4f}\n\n")

    lr_results = pd.concat([lr_results, pd.DataFrame([{"horizon": h, "accuracy": test_acc}])], ignore_index=True)

print(lr_results)

Tuning hyperparameters of logistic regression for horizon 1...
horizon=1, C=0.01 => accuracy=0.5844
horizon=1, C=0.1 => accuracy=0.5795
horizon=1, C=1 => accuracy=0.5710
horizon=1, C=10 => accuracy=0.5703
horizon=1, C=100 => accuracy=0.5703
horizon=1, C=1000 => accuracy=0.5703
For horizon: 1, best params on validation: {'C': 0.01} with accuracy: 0.5843881856540084
Logistic Regression model training for horizon: 1...
Finished using Logistic Regression model for horizon: 1! Training time: 0.0355s, Accuracy: 0.5650


Tuning hyperparameters of logistic regression for horizon 6...


/tmp/ipykernel_31287/2136399920.py:116: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  lr_results = pd.concat([lr_results, pd.DataFrame([{"horizon": h, "accuracy": test_acc}])], ignore_index=True)


horizon=6, C=0.01 => accuracy=0.4205
horizon=6, C=0.1 => accuracy=0.4395
horizon=6, C=1 => accuracy=0.4423
horizon=6, C=10 => accuracy=0.4402
horizon=6, C=100 => accuracy=0.4409
horizon=6, C=1000 => accuracy=0.4409
For horizon: 6, best params on validation: {'C': 1} with accuracy: 0.44233473980309423
Logistic Regression model training for horizon: 6...
Finished using Logistic Regression model for horizon: 6! Training time: 0.0702s, Accuracy: 0.5207


Tuning hyperparameters of logistic regression for horizon 12...
horizon=12, C=0.01 => accuracy=0.5084
horizon=12, C=0.1 => accuracy=0.5183
horizon=12, C=1 => accuracy=0.5218
horizon=12, C=10 => accuracy=0.5218
horizon=12, C=100 => accuracy=0.5218
horizon=12, C=1000 => accuracy=0.5218
For horizon: 12, best params on validation: {'C': 1} with accuracy: 0.5218002812939522
Logistic Regression model training for horizon: 12...
Finished using Logistic Regression model for horizon: 12! Training time: 0.0572s, Accuracy: 0.4841


Tuning hyperparame

## Naive Baseline

In [46]:
naive_baseline_results = pd.DataFrame([], columns = ["horizon", "accuracy"])
for h in horizons:
    y_h_test = test[f"CO_fut_{h}h_class"].iloc[:(-1 * h)]
    nb_pred = test["CO_class"].iloc[:(-1 * h)]
    test_acc = accuracy_score(y_h_test, nb_pred)
    print(f"Finished calculating naive baseline accuracy for horizon: {h}! Accuracy: {test_acc:.4f}\n\n")

    naive_baseline_results = pd.concat([naive_baseline_results, pd.DataFrame([{"horizon": h, "accuracy": test_acc}])], ignore_index=True)

print(naive_baseline_results)
    

Finished calculating naive baseline accuracy for horizon: 1! Accuracy: 0.7293


Finished calculating naive baseline accuracy for horizon: 6! Accuracy: 0.3815


Finished calculating naive baseline accuracy for horizon: 12! Accuracy: 0.3736


Finished calculating naive baseline accuracy for horizon: 24! Accuracy: 0.5668


  horizon  accuracy
0       1  0.729297
1       6  0.381526
2      12  0.373602
3      24  0.566802


/tmp/ipykernel_31287/2186062848.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  naive_baseline_results = pd.concat([naive_baseline_results, pd.DataFrame([{"horizon": h, "accuracy": test_acc}])], ignore_index=True)


# Results

In [47]:
overall_results = pd.DataFrame([], columns = ["horizon", "decision_tree_accuracy", "svm_accuracy", "logistic_regression_accuracy", "naive_baseline_accuracy"])

for h in horizons:
    h_res = {
        "horizon": h,
        "decision_tree_accuracy": dt_results.loc[dt_results["horizon"] == h, "accuracy"].iloc[0],
        "svm_accuracy": svm_results.loc[svm_results["horizon"] == h, "accuracy"].iloc[0],
        "logistic_regression_accuracy": lr_results.loc[lr_results["horizon"] == h, "accuracy"].iloc[0],
        "naive_baseline_accuracy": naive_baseline_results.loc[naive_baseline_results["horizon"] == h, "accuracy"].iloc[0]
    }
    overall_results = pd.concat([overall_results, pd.DataFrame([h_res])], ignore_index=True)

print(overall_results)

  horizon  decision_tree_accuracy  svm_accuracy  logistic_regression_accuracy  \
0       1                0.604185      0.512021                      0.565004   
1       6                0.515841      0.342258                      0.520750   
2      12                0.550336      0.349888                      0.484116   
3      24                0.557355      0.557355                      0.473234   

   naive_baseline_accuracy  
0                 0.729297  
1                 0.381526  
2                 0.373602  
3                 0.566802  


/tmp/ipykernel_31287/2763215419.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  overall_results = pd.concat([overall_results, pd.DataFrame([h_res])], ignore_index=True)
